In [2]:
# 학습된 가중치 매개변수
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from layer import *


class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망
    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [ ]:
# 훈련용 코드
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from mnist import load_mnist

from trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보관
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

train loss:2.3112022407206805
=== epoch:1, train acc:0.088, test acc:0.096 ===
train loss:2.2786109733748416
train loss:2.313928288455426
train loss:2.2942072394793787
train loss:2.298415662549137
train loss:2.2578003143131276
train loss:2.271179934879758
train loss:2.299687051624284
train loss:2.275271211266005
train loss:2.273287044132027
train loss:2.2454297178471694
train loss:2.272756200633782
train loss:2.290254198811464
train loss:2.251587496946292
train loss:2.2428105659201183
train loss:2.249216787397556
train loss:2.2411361828072986
train loss:2.25741606715429
train loss:2.198990091807112
train loss:2.1893470658783944
train loss:2.1837183287206177
train loss:2.1686512050808657
train loss:2.191055018835646
train loss:2.129059395603528
train loss:2.1341818650446185
train loss:2.0781336886859756
train loss:2.101825544465716
train loss:2.06537145590717
train loss:2.1009854736973863
train loss:2.136986531051401
train loss:2.1343158706639698
train loss:2.0097316009377026
train loss

train loss:1.3697905138598556
train loss:1.4661519973997161
train loss:1.2980817458305862
train loss:1.3282166144776042
train loss:1.155206961502526
train loss:1.191033721932843
train loss:1.2016863448043094
train loss:1.2156137715309816
train loss:1.112579679326715
train loss:1.0791409968838779
train loss:1.1995979515921877
train loss:1.1618283477173283
train loss:1.4544675195896937
train loss:1.1687169448198693
train loss:1.3683962277378883
train loss:1.2493714860006762
train loss:1.1592577538516486
train loss:1.1711921314107785
train loss:1.210845997102705
train loss:1.1951809235930857
train loss:1.2941035734535098
train loss:1.1697011494284042
train loss:1.1968292694073803
train loss:1.1999396059065757
train loss:1.285775174391612
train loss:1.095110710605791
train loss:1.0620912013420591
train loss:1.2228522149775514
train loss:0.9836593469890927
train loss:1.050023131022546
train loss:1.1613015745822093
train loss:1.0510200159722065
train loss:1.2220409474293696
train loss:1.1134

train loss:0.9541611989340385
train loss:1.1597223431592456
train loss:1.1543819519123877
train loss:1.086466207237675
train loss:1.0351191745070876
train loss:1.0606122972784222
train loss:0.9578277352604994
train loss:1.2104970626148104
train loss:1.172196991294953
train loss:0.9507700514571609
train loss:1.033136652079613
train loss:1.147212575236419
train loss:0.9221646183074524
train loss:1.0663935786970162
train loss:1.0711287542574937
train loss:1.1062206132235406
train loss:0.9389557648087754
train loss:0.9650387712462983
train loss:1.1548966931462874
train loss:0.8642784850466989
train loss:1.028146291911476
train loss:0.9576659842070124
train loss:1.1809253027003843
train loss:1.058219129816411
train loss:1.0418468208577756
train loss:1.0083774315746996
train loss:1.1861525193673457
train loss:1.0055689156597998
train loss:0.9116395704556238
train loss:1.0029057452652324
train loss:0.9625801985212263
train loss:1.0263093500618008
train loss:0.9995432040299922
train loss:1.149

train loss:1.1334486257785654
train loss:0.9909732830260718
train loss:1.0445970232274036
train loss:1.2427346046940055
train loss:1.0802424792793384
train loss:1.1411559735436345
train loss:0.9107106946235541
train loss:0.8496718913832395
train loss:1.1376522225057735
train loss:0.8949305180096312
train loss:0.9034478979697512
train loss:1.0320231134683089
train loss:0.9132604596411091
train loss:0.9570107906830903
train loss:0.9574828482965141
train loss:0.9279576762328173
train loss:1.094839405177333
train loss:0.9426065827266512
train loss:1.0193231459463088
train loss:1.0763126515643493
train loss:1.119385946208728
train loss:0.8851779792607491
train loss:1.0072652679984435
train loss:0.9699129490388796
train loss:1.0342211400211638
train loss:1.0818603163057694
train loss:0.9383049992801059
train loss:1.0509963831869322
train loss:0.9356203182649174
train loss:1.0679979154083628
train loss:1.144373955883391
train loss:0.9425188242324803
train loss:0.9540872405460326
train loss:1.